##### Чтение данных target_class и выбор только нужных строки, поподающие в одну из выборок и проставляем метки кред и фрод дефолта

In [ ]:
def target_filter(target_data):
    target_data.rename(columns={'credit_risk':'экспертный_кред_дефолт'}, inplace=True)
    target_data['экспертный_кред_дефолт'] = target_data['экспертный_кред_дефолт'].astype('int')

    target_data['Метка кред дефолта'] = target_data['Метка кред дефолта'].combine_first(target_data['экспертный_кред_дефолт'])
    # target_data['Метка фрод дефолта'] = target_data['combined_fraud_default']

    m1 = (target_data['Попадает в выборку']=='обе модели')
    m2 = (target_data['Попадает в выборку']=='только фрод')
    target_data = target_data.loc[m1|m2]

    #  возьмем только данные с даты принятия решения от 2020 года
    target_data = target_data[target_data['Дата принятия решения по сделке']>'01.01.2020'].copy()

    #  проверка что в 1 сделке только одни ИНН участвует
    target_data['ИНН_count'] = target_data.groupby('Номер сделки')['ИНН контрагента'].transform('nunique')
    Check_length(len(target_data[target_data['ИНН_count']>1]), 0) 

    target_data.reset_index(inplace=True)
    target_data.rename(columns = {'index':'Приложение'}, inplace=True)

    keep_columns = ['Приложение', 'Номер сделки', 'Номер договора (кратко)', 'ИНН контрагента', 
                    'Дата принятия решения по сделке', 
                    'Метка фрод дефолта', 'Метка кред дефолта', 'Попадает в выборку']

    target_data = target_data[keep_columns]

    return target_data   

In [ ]:
#  исключить крупных клиентов ( по сумма сделки финанисрования) из выборки

def filterMajorClients(df_target):
    
    m = (df_target['DATE_ACTUAL'].between('03.12.18', '25.01.21')) & ((df_target['NL_NEW_PRODUCTNAME']=='Express') | (df_target['O_NEW_TOTALFINANCINGSUM']<=10e+6))
    df_target.loc[m, 'маршрутизация_по_моделям'] = True              
    m = (df_target['DATE_ACTUAL'].between('25.01.21', '15.12.21')) & ((df_target['NL_NEW_PRODUCTNAME']=='Express') | (df_target['O_NEW_TOTALFINANCINGSUM']<=15e+6))
    df_target.loc[m, 'маршрутизация_по_моделям'] = True
    m = (df_target['DATE_ACTUAL'].between('15.12.21', '15.07.2022')) & ((df_target['NL_NEW_PRODUCTNAME']=='Express') | (df_target['O_NEW_TOTALFINANCINGSUM']<=20313824))
    df_target.loc[m, 'маршрутизация_по_моделям'] = True
    m = (df_target['DATE_ACTUAL'].between('15.07.22', '31.03.2023')) & ((df_target['NL_NEW_PRODUCTNAME']=='Express' )| (df_target['O_NEW_TOTALFINANCINGSUM']<=26711533))
    df_target.loc[m, 'маршрутизация_по_моделям'] = True
    m = (df_target['DATE_ACTUAL']>'31.03.2023') & ((df_target['NL_NEW_PRODUCTNAME']=='Express') | (df_target['O_NEW_TOTALFINANCINGSUM']<=5e+7))
    df_target.loc[m, 'маршрутизация_по_моделям'] = True
    
    df_target.loc[(df_target['маршрутизация_по_моделям']!=True), 'Попадает в выборку'] = None
    
    return df_target

##### Класс Target
    Класс Target служит для того, чтобы хранить data данные, а также информацию, являются ли они фрод или кред, и самому формировать эти данные из массива target-class данных
    Пример использования:
    target_frod = Target(type='фрод', data=target_features) # это фрод дефолт, макс по обращению

    target_frod.print_param() # печатает основные показатели обучающих данных (сколько меток)


In [ ]:
class Target():
    def __init__(self, type, target_data, removeMajorClients=False, by_application=True):
        
        target_data = target_data.copy()
        self.type = type # Фрод или кред
        self.mark = 'Метка {} дефолта'.format(type) # наименования столбца в талице target_class

        """ by_application = True метка будет агрегироваться по максимуму в рамках одной сделки
            by_application = False это приведенный фрод дефолт на самый первое обращение
        """
        self.by_application = by_application

        if type == 'фрод':
            valid = ["только фрод", "обе модели"]
        elif type == 'кред':
            valid = ["обе модели"]
            
        if removeMajorClients: 
            target_data = filterMajorClients(target_data)
        
        if by_application: # метка будет агрегироваться по максимуму в рамках одной сделки
            mask = (target_data['ИНН контрагента'].notnull())

            data = target_data[mask & (target_data['Попадает в выборку'].isin(valid))].reset_index(drop=True)

            # берем по обращению метку максимальную
            data['Метка дефолта'] =  data.groupby(['Номер сделки', 'ИНН контрагента'])[self.mark].transform(max)

            # и удалим дубли по номеру сделки. Это наши рассматриваемые таблица без дублей по номерам сделок
            data = data.drop_duplicates(subset=['Номер сделки', 'ИНН контрагента']).reset_index()
            
        else:  #  это приведенный фрод дефолт на самый первое обращение
            mask = (target_data['ИНН контрагента'].notnull())
  
            data = target_data[mask & (target_data['Попадает в выборку'].isin(valid))].reset_index(drop=True)
            
            # берем по контрагенту метку максимальную
            data['Метка дефолта'] = data.groupby('ИНН контрагента')[self.mark].transform(max) 

            # уберем все остальные сделки по фроду кроме первого обращения
            data['Num_Application'] = data.sort_values(by='Дата принятия решения по сделке').groupby('ИНН контрагента').cumcount()
            mask = ((data['Num_Application']>0) & (data['Метка дефолта']==1))
            data = data[~mask].reset_index(drop=True)
            data.drop(columns='Num_Application', inplace=True)

            # и удалим дубли по номеру сделки
            data.drop_duplicates(subset=['Номер сделки', 'ИНН контрагента'], inplace=True, ignore_index=True)
            data = data.reset_index()
        
        # результирующая таблица с метками для дальнейшего анализа
        self.data = data # Это наша рассматриваемая таблица без дублей по номерам сделок

    def print_param(self):        # печатаем, что получилось
        type_app = 'по обращениям' if self.by_application else 'приведенный фрод дефолт на самое первое обращение' 

        print("\nПараметры обработанной таблицы портфеля рисков  {}:".format(type_app))
        print("\t{} дефолт = 1  у выборки {} из {} ({}%)".format(self.type, self.data['Метка дефолта'].sum(), len(self.data), 
                                round(float(self.data['Метка дефолта'].sum()/len(self.data)*100), 2)))